https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-indepth.html
    
# Predicting Columns in a Table - In Depth: porto_seguro (classification)

Tip: If you are new to AutoGluon, review "Predicting Columns in a Table - Quick Start" to learn the basics of the AutoGluon API.

https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-quickstart.html#sec-tabularquick

This tutorial describes how you can exert greater control when using AutoGluon’s `fit()` or `predict()`. Recall that to maximize predictive performance, you should always first try `fit()` with all default arguments except `eval_metric` and `presets`, before you experiment with other arguments covered in this in-depth tutorial like `hyperparameter_tune_kwargs`, `hyperparameters`, `num_stack_levels`, `num_bag_folds`, `num_bag_sets`, etc.

Using the same census data table as in the "Predicting Columns in a Table - Quick Start" tutorial, we’ll now predict the occupation of an individual - a multiclass classification problem. Start by importing AutoGluon’s `TabularPredictor` and `TabularDataset`, and loading the data.

In [53]:
from autogluon.tabular import TabularDataset, TabularPredictor

import pandas as pd
import numpy as np

In [2]:
# train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data = TabularDataset("porto_train.csv")
test_data = TabularDataset("porto_test.csv")
# subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
# train_data = train_data.sample(n=subsample_size, random_state=0)

In [3]:
train_data.info()

<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 476170 entries, 0 to 476169
Data columns (total 60 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              476170 non-null  int64  
 1   target          476170 non-null  int64  
 2   ps_ind_01       476170 non-null  int64  
 3   ps_ind_02_cat   476170 non-null  int64  
 4   ps_ind_03       476170 non-null  int64  
 5   ps_ind_04_cat   476170 non-null  int64  
 6   ps_ind_05_cat   476170 non-null  int64  
 7   ps_ind_06_bin   476170 non-null  int64  
 8   ps_ind_07_bin   476170 non-null  int64  
 9   ps_ind_08_bin   476170 non-null  int64  
 10  ps_ind_09_bin   476170 non-null  int64  
 11  ps_ind_10_bin   476170 non-null  int64  
 12  ps_ind_11_bin   476170 non-null  int64  
 13  ps_ind_12_bin   476170 non-null  int64  
 14  ps_ind_13_bin   476170 non-null  int64  
 15  ps_ind_14       476170 non-null  int64  
 16  ps_ind_15       476170 non-null  int64  
 17  

In [4]:
test_data.info()

<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 119042 entries, 0 to 119041
Data columns (total 59 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              119042 non-null  int64  
 1   target          119042 non-null  int64  
 2   ps_ind_01       119042 non-null  int64  
 3   ps_ind_02_cat   119042 non-null  int64  
 4   ps_ind_03       119042 non-null  int64  
 5   ps_ind_04_cat   119042 non-null  int64  
 6   ps_ind_05_cat   119042 non-null  int64  
 7   ps_ind_06_bin   119042 non-null  int64  
 8   ps_ind_07_bin   119042 non-null  int64  
 9   ps_ind_08_bin   119042 non-null  int64  
 10  ps_ind_09_bin   119042 non-null  int64  
 11  ps_ind_10_bin   119042 non-null  int64  
 12  ps_ind_11_bin   119042 non-null  int64  
 13  ps_ind_12_bin   119042 non-null  int64  
 14  ps_ind_13_bin   119042 non-null  int64  
 15  ps_ind_14       119042 non-null  int64  
 16  ps_ind_15       119042 non-null  int64  
 17  

In [5]:
train_data.head().T

,0,1,2,3,4
id,9.000000,13.000000,16.000000,17.000000,20.000000
target,0.000000,0.000000,0.000000,0.000000,0.000000
ps_ind_01,1.000000,5.000000,0.000000,0.000000,2.000000
ps_ind_02_cat,1.000000,4.000000,1.000000,2.000000,1.000000
ps_ind_03,7.000000,9.000000,2.000000,0.000000,3.000000
ps_ind_04_cat,0.000000,1.000000,0.000000,1.000000,1.000000
ps_ind_05_cat,0.000000,0.000000,0.000000,0.000000,0.000000
ps_ind_06_bin,0.000000,0.000000,1.000000,1.000000,0.000000
ps_ind_07_bin,0.000000,0.000000,0.000000,0.000000,1.000000
ps_ind_08_bin,1.000000,1.000000,0.000000,0.000000,0.000000


In [6]:
label = 'target'
print("Summary of target column: \n", train_data[label].describe())

Summary of target column: 
 count    476170.000000
mean          0.036455
std           0.187421
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: target, dtype: float64


## Data Cleaning

In [7]:
train_data = train_data.replace(-1, np.nan)
test_data = test_data.replace(-1, np.nan)

In [8]:
train_data = train_data.drop(["id", "fold"], axis=1)
test_data = test_data.drop(["id"], axis=1)

In [9]:
train_data.shape

(476170, 58)

In [10]:
test_data.shape

(119042, 58)

In [11]:
cat_vars = [col for col in train_data.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [12]:
for col in cat_vars:
    test_data[col] = test_data[col].astype('category')
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_data[col] = train_data[col].astype('category')

In [13]:
# new_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
# test_data = new_data[5000:].copy()  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
# val_data = new_data[:5000].copy()

# metric = 'accuracy' # we specify eval-metric just for demo (unnecessary as it's the default)
metric = 'roc_auc' # we specify eval-metric just for demo (unnecessary as it's the default)

## Specifying hyperparameters and tuning them

We first demonstrate hyperparameter-tuning and how you can provide your own validation dataset that AutoGluon internally relies on to: tune hyperparameters, early-stop iterative training, and construct model ensembles. One reason you may specify validation data is when future test data will stem from a different distribution than training data (and your specified validation data is more representative of the future data that will likely be encountered).

If you don’t have a strong reason to provide your own validation dataset, we recommend you omit the `tuning_data` argument. This lets AutoGluon automatically select validation data from your provided training set (it uses smart strategies such as stratified sampling). For greater control, you can specify the `holdout_fra`c argument to tell AutoGluon what fraction of the provided training data to hold out for validation.

**Caution:** Since AutoGluon tunes internal knobs based on this validation data, performance estimates reported on this data may be over-optimistic. For unbiased performance estimates, you should always call `predict()` on a separate dataset (that was never passed to `fit()`), as we did in the previous Quick-Start tutorial. We also emphasize that most options specified in this tutorial are chosen to minimize runtime for the purposes of demonstration and you should select more reasonable values in order to obtain high-quality models.

`fit()` trains neural networks and various types of tree ensembles by default. You can specify various hyperparameter values for each type of model. For each hyperparameter, you can either specify a single fixed value, or a search space of values to consider during hyperparameter optimization. Hyperparameters which you do not specify are left at default settings chosen automatically by AutoGluon, which may be fixed values or search spaces.

In [14]:
import autogluon.core as ag

In [15]:
# specifies non-default hyperparameter values for neural network models
#    num_epochs: number of training epochs (controls training time of NN models)
#    learning_rate: learning rate used in training (real-valued hyperparameter searched on log-scale)
#    activation: activation function used in NN (categorical hyperparameter, default = first entry)
#    layers: each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
#    dropout_prob: dropout probability (real-valued hyperparameter)
nn_options = {
    'num_epochs': 10,
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1)
}

nn_options

{'num_epochs': 10,
 'learning_rate': Real: lower=0.0001, upper=0.01,
 'activation': Categorical['relu', 'softrelu', 'tanh'],
 'layers': Categorical[[100], [1000], [200, 100], [300, 200, 100]],
 'dropout_prob': Real: lower=0.0, upper=0.5}

In [16]:
# specifies non-default hyperparameter values for lightGBM gradient boosted trees
#    num_boost_round: number of boosting rounds (controls training time of GBM models)
#    num_leaves: number of leaves in trees (integer hyperparameter)
gbm_options = {  
    'num_boost_round': 100,
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36)
}

gbm_options

{'num_boost_round': 100, 'num_leaves': Int: lower=26, upper=66}

In [17]:
# hyperparameters of each model type
# When these keys are missing from hyperparameters dict, no models of that type are trained
#    NN: NOTE: comment this line out if you get errors on Mac OSX
hyperparameters = {  
    'GBM': gbm_options,
    'NN': nn_options
}

hyperparameters

{'GBM': {'num_boost_round': 100, 'num_leaves': Int: lower=26, upper=66},
 'NN': {'num_epochs': 10,
  'learning_rate': Real: lower=0.0001, upper=0.01,
  'activation': Categorical['relu', 'softrelu', 'tanh'],
  'layers': Categorical[[100], [1000], [200, 100], [300, 200, 100]],
  'dropout_prob': Real: lower=0.0, upper=0.5}}

In [18]:
# train various models for ~2 min
# time_limit = 2*60

# https://www.kaggle.com/code/daikikatsuragawa/tps-mar-2021-benchmark-using-autogluon/notebook
# If the value of time_limit is too small, we will get the following error:
# ValueError: AutoGluon did not successfully train any models

time_limit = 60*60
time_limit

3600

In [19]:
# try at most 5 different hyperparameter configurations for each type of model
num_trials = 5
num_trials

5

In [20]:
# to tune hyperparameters using Bayesian optimization routine with a local scheduler
search_strategy = 'auto'
search_strategy

'auto'

In [21]:
# HPO is not performed unless hyperparameter_tune_kwargs is specified
hyperparameter_tune_kwargs = {
    'num_trials': num_trials,
    'searcher': search_strategy
}

hyperparameter_tune_kwargs

{'num_trials': 5, 'searcher': 'auto'}

In [22]:
label

'target'

In [23]:
metric

'roc_auc'

In [24]:
# NEED TO DEBUG
"""
predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    tuning_data=val_data,
    time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
)
"""

'\npredictor = TabularPredictor(label=label, eval_metric=metric).fit(\n    train_data,\n    tuning_data=val_data,\n    time_limit=time_limit,\n    hyperparameters=hyperparameters,\n    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs\n)\n'

In [25]:
predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    time_limit=time_limit
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230702_040607/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230702_040607/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    168131.98 MB
	T

We again demonstrate how to use the trained models to predict on the test data.

In [26]:
# y_pred = predictor.predict(test_data_nolabel)
# AssertionError: `evaluate_predictions` requires y_pred_proba input when evaluating "roc_auc"... 
# Please generate valid input via `predictor.predict_proba(data)`.
y_pred = predictor.predict_proba(test_data_nolabel)

In [27]:
# print("Predictions:  ", list(y_pred)[:5])
print("Predictions:\n", y_pred[:5])

Predictions:
           0         1
0  0.952504  0.047496
1  0.955137  0.044863
2  0.972644  0.027356
3  0.982837  0.017163
4  0.978452  0.021548


In [28]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: roc_auc on test data: 0.6318303042376591
Evaluations on test data:
{
    "roc_auc": 0.6318303042376591
}


Use the following to view a summary of what happened during fit. Now this command will show details of the hyperparameter-tuning process for each type of model:

In [29]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.612350       0.352664  605.562781                0.001182           1.260499            2       True         14
1              CatBoost   0.602004       0.024931   47.651210                0.024931          47.651210            1       True          7
2        ExtraTreesEntr   0.601840       0.167636    9.599753                0.167636           9.599753            1       True          9
3               XGBoost   0.601470       0.037694    7.182353                0.037694           7.182353            1       True         11
4        NeuralNetTorch   0.594228       0.057916  237.406210                0.057916         237.406210            1       True         12
5            LightGBMXT   0.591881       0.014627    3.995357                0.014627           3.

In the above example, the predictive performance may be poor because we specified very little training to ensure quick runtimes. You can call `fit()` multiple times while modifying the above settings to better understand how these choices affect performance outcomes. For example: you can comment out the `train_data.head` command or increase `subsample_size` to train using a larger dataset, increase the `num_epochs` and `num_boost_round` hyperparameters, and increase the `time_limit` (which you should do for all code in these tutorials). To see more detailed output during the execution of `fit()`, you can also pass in the argument: `verbosity=3`.

## Model ensembling with stacking/bagging

Beyond hyperparameter-tuning with a correctly-specified evaluation metric, two other methods to boost predictive performance are bagging and stack-ensembling:

"AutoGluon-Tabular: Robust and Accurate AutoML for Structured Data": https://arxiv.org/abs/2003.06505

You’ll often see performance improve if you specify `num_bag_folds = 5-10, num_stack_levels = 1-3` in the call to `fit()`, but this will increase training times and memory/disk usage.

In [30]:
# last  argument is just for quick demo here, omit it in real applications
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,                                                              
    # hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230702_041736/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230702_041736/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> l

	0.09s	 = Validation runtime
AutoGluon training complete, total runtime = 2433.87s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230702_041736/")


You should not provide `tuning_data` when stacking/bagging, and instead provide all your available data as `train_data` (which AutoGluon will split in more intellgent ways). `num_bag_sets` controls how many times the k-fold bagging process is repeated to further reduce variance (increasing this may further boost accuracy but will substantially increase training times, inference latency, and memory/disk usage). Rather than manually searching for good bagging/stacking values yourself, AutoGluon will automatically select good values for you if you specify `auto_stack` instead:

In [31]:
save_path = 'agModels-predict_porto_2'  # folder where to store trained models

# last 2 arguments are for quick demo, omit them in real applications
predictor = TabularPredictor(label=label, eval_metric=metric, path=save_path).fit(
    train_data, 
    auto_stack=True,
    time_limit=30, 
    hyperparameters={'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}} 
)

Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "agModels-predict_porto_2/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    154564.05 MB
	Tra

Often stacking/bagging will produce superior accuracy than hyperparameter-tuning, but you may try combining both techniques (note: specifying `presets='best_quality'` in `fit()` simply sets `auto_stack=True`).

## Prediction options (inference)

Even if you’ve started a new Python session since last calling `fit()`, you can still load a previously trained predictor from disk:

```
# `predictor.path` is another way to get the relative path needed to later load predictor.
predictor = TabularPredictor.load(save_path)  
```

Above `save_path` is the same folder previously passed to `TabularPredictor`, in which all the trained models have been saved. You can train easily models on one machine and deploy them on another. Simply copy the `save_path` folder to the new machine and specify its new path in `TabularPredictor.load()`.

We can make a prediction on an individual example rather than a full dataset:

In [32]:
datapoint = test_data_nolabel.iloc[[0]]  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame

print(datapoint)
predictor.predict(datapoint)

2023-07-02 04:58:31,848	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-02 04:58:31,849	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-02 04:58:31,853	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-02 04:58:31,854	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-02 04:58:31,856	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UN

   ps_ind_01 ps_ind_02_cat  ps_ind_03 ps_ind_04_cat ps_ind_05_cat  \
0          2           2.0          5           1.0           0.0   

   ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  ps_ind_09_bin  ps_ind_10_bin  \
0              0              1              0              0              0   

   ...  ps_calc_11  ps_calc_12  ps_calc_13  ps_calc_14  ps_calc_15_bin  \
0  ...           9           1           5           8               0   

   ps_calc_16_bin  ps_calc_17_bin  ps_calc_18_bin  ps_calc_19_bin  \
0               1               1               0               0   

   ps_calc_20_bin  
0               1  

[1 rows x 57 columns]


0    0
Name: target, dtype: object

To output predicted class probabilities instead of predicted classes, you can use:

In [33]:
predictor.predict_proba(datapoint)  # returns a DataFrame that shows which probability corresponds to which class

,0,1
0,0.956901,0.043099


By default, `predict()` and `predict_proba()` will utilize the model that AutoGluon thinks is most accurate, which is usually an ensemble of many individual models. Here’s how to see which model this is:

In [34]:
predictor.get_model_best()

'WeightedEnsemble_L2'

We can instead specify a particular model to use for predictions (e.g. to reduce inference latency). Note that a ‘model’ in AutoGluon may refer to for example a single Neural Network, a bagged ensemble of many Neural Network copies trained on different training/validation splits, a weighted ensemble that aggregates the predictions of many other models, or a stacker model that operates on predictions output by other models. This is akin to viewing a Random Forest as one ‘model’ when it is in fact an ensemble of many decision trees.

Before deciding which model to use, let’s evaluate all of the models AutoGluon has previously trained on our test data:

In [35]:
# predictor.leaderboard(test_data, silent=True)
predictor.leaderboard(test_data, silent=True).T

,0,1
model,LightGBM_BAG_L1,WeightedEnsemble_L2
score_test,0.618328,0.618328
score_val,0.623974,0.623974
pred_time_test,0.762287,0.767083
pred_time_val,0.744501,0.834036
fit_time,4.017492,4.108663
pred_time_test_marginal,0.762287,0.004795
pred_time_val_marginal,0.744501,0.089535
fit_time_marginal,4.017492,0.091171
stack_level,1,2


The leaderboard shows each model’s predictive performance on the test data (`score_test`) and validation data (`score_val`), as well as the time required to: produce predictions for the test data (`pred_time_val`), produce predictions on the validation data (`pred_time_val`), and train only this model (`fit_time`). Below, we show that a leaderboard can be produced without new data (just uses the data previously reserved for validation inside `fit`) and can display extra information about each model:

In [36]:
# predictor.leaderboard(extra_info=True, silent=True)
predictor.leaderboard(extra_info=True, silent=True).T

,0,1
model,LightGBM_BAG_L1,WeightedEnsemble_L2
score_val,0.623974,0.623974
pred_time_val,0.744501,0.834036
fit_time,4.017492,4.108663
pred_time_val_marginal,0.744501,0.089535
fit_time_marginal,4.017492,0.091171
stack_level,1,2
can_infer,True,True
fit_order,1,2
num_features,57,1


The expanded leaderboard shows properties like how many features are used by each model (`num_features`), which other models are ancestors whose predictions are required inputs for each model (`ancestors`), and how much memory each model and all its ancestors would occupy if simultaneously persisted (`memory_size_w_ancestors`). See the leaderboard documentation for full details:

https://auto.gluon.ai/0.1.0/api/autogluon.predictor.html#autogluon.tabular.TabularPredictor.leaderboard

Here’s how to specify a particular model to use for prediction instead of AutoGluon’s default model-choice:

In [37]:
i = 0  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

Prediction from LightGBM_BAG_L1 model: 0


We can easily access various information about the trained predictor or a particular model:

In [38]:
all_models = predictor.get_model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

The `predictor` also remembers what metric predictions should be evaluated with, which can be done with ground truth labels as follows:

In [40]:
# y_pred = predictor.predict(test_data_nolabel)
y_pred = predictor.predict_proba(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6183275066932732
Evaluations on test data:
{
    "roc_auc": 0.6183275066932732,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


However, you must be careful here as certain metrics require predicted probabilities rather than classes. Since the label columns remains in the `test_data` DataFrame, we can instead use the shorthand:

In [41]:
perf = predictor.evaluate(test_data)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6183275066932732
Evaluations on test data:
{
    "roc_auc": 0.6183275066932732,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


which will correctly select between `predict()` or `predict_proba()` depending on the evaluation metric.

## Interpretability (feature importance)

To better understand our trained predictor, we can estimate the overall importance of each feature:

In [42]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 57 features using 5000 rows with 5 shuffle sets...
	41.09s	= Expected runtime (8.22s per shuffle set)
	12.58s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
ps_ind_05_cat,2.336078e-02,0.007582,0.001163,5,0.038971,0.007750
ps_car_13,7.689422e-03,0.011673,0.107378,5,0.031725,-0.016346
ps_reg_03,7.202950e-03,0.002470,0.001429,5,0.012289,0.002117
ps_car_07_cat,4.274511e-03,0.007997,0.149022,5,0.020741,-0.012192
ps_ind_03,3.842139e-03,0.002970,0.022226,5,0.009958,-0.002274
ps_ind_17_bin,3.671786e-03,0.004612,0.074835,5,0.013169,-0.005825
ps_reg_01,3.067102e-03,0.002393,0.022816,5,0.007993,-0.001859
ps_ind_15,2.409103e-03,0.001559,0.012969,5,0.005620,-0.000801
ps_car_01_cat,2.277008e-03,0.004534,0.162147,5,0.011613,-0.007059
ps_ind_06_bin,1.987626e-03,0.005054,0.214437,5,0.012395,-0.008419


Computed via permutation-shuffling, these feature importance scores quantify the drop in predictive performance (of the already trained predictor) when one column’s values are randomly shuffled across rows. 

https://explained.ai/rf-importance/

The top features in this list contribute most to AutoGluon’s accuracy (for predicting when/if a patient will be readmitted to the hospital). Features with non-positive importance score hardly contribute to the predictor’s accuracy, or may even be actively harmful to include in the data (consider removing these features from your data and calling `fit` again). These scores facilitate interpretability of the predictor’s global behavior (which features it relies on for all predictions) rather than local explanations that only rationalize one particular prediction.

https://christophm.github.io/interpretable-ml-book/taxonomy-of-interpretability-methods.html

## Accelerating inference

We describe multiple ways to reduce the time it takes for AutoGluon to produce predictions.

### Keeping models in memory

By default, AutoGluon loads models into memory one at a time and only when they are needed for prediction. This strategy is robust for large stacked/bagged ensembles, but leads to slower prediction times. If you plan to repeatedly make predictions (e.g. on new datapoints one at a time rather than one large test dataset), you can first specify that all models required for inference should be loaded into memory as follows:

In [54]:
predictor.persist_models()

num_test = 20
# preds = np.array([''] * num_test, dtype='object')
preds = pd.DataFrame()
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    # pred_numpy = predictor.predict(datapoint, as_pandas=False)
    pred_numpy = predictor.predict_proba(datapoint, as_pandas=True)
    # preds[i] = pred_numpy[0]
    preds = preds.append(pred_numpy)  # use "pd.concat"

# perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
perf = predictor.evaluate_predictions(y_true=y_test[:num_test], y_pred=preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist_models()  # free memory by clearing models, future predict() calls will load models from disk

The following 2 models were already persisted and will be ignored in the model loading process: ['LightGBM_BAG_L1', 'WeightedEnsemble_L2']
No valid unpersisted models were specified to be persisted, so no change in model persistence was performed.
/tmp/ipykernel_35752/3590748705.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(pred_numpy)
/tmp/ipykernel_35752/3590748705.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(pred_numpy)
/tmp/ipykernel_35752/3590748705.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(pred_numpy)
/tmp/ipykernel_35752/3590748705.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

Predictions:             0         1
0   0.956901  0.043099
1   0.963648  0.036352
2   0.968904  0.031096
3   0.975422  0.024578
4   0.971613  0.028387
5   0.971882  0.028118
6   0.971016  0.028984
7   0.962172  0.037828
8   0.969281  0.030719
9   0.969197  0.030803
10  0.971344  0.028656
11  0.970507  0.029493
12  0.966494  0.033506
13  0.939654  0.060346
14  0.959474  0.040526
15  0.973707  0.026293
16  0.965323  0.034677
17  0.959917  0.040083
18  0.974798  0.025202
19  0.959847  0.040153


['LightGBM_BAG_L1', 'WeightedEnsemble_L2']

You can alternatively specify a particular model to persist via the models argument of `persist_models()`, or simply set `models='all'` to simultaneously load every single model that was trained during `fit`.

## Using smaller ensemble or faster model for prediction

Without having to retrain any models, one can construct alternative ensembles that aggregate individual models’ predictions with different weighting schemes. These ensembles become smaller (and hence faster for prediction) if they assign nonzero weight to less models. You can produce a wide variety of ensembles with different accuracy-speed tradeoffs like this:

In [55]:
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True, silent=True)

Fitting model: WeightedEnsemble_L2Best ...
	0.624	 = Validation score   (roc_auc)
	0.1s	 = Training   runtime
	0.09s	 = Validation runtime


Alternative ensembles you can use for prediction: ['WeightedEnsemble_L2Best']


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.623974,0.744501,4.017492,0.744501,4.017492,1,True,1


The resulting leaderboard will contain the most accurate model for a given inference-latency. You can select whichever model exhibits acceptable latency from the leaderboard and use it for prediction.

In [56]:
model_for_prediction = additional_ensembles[0]
# predictions = predictor.predict(test_data, model=model_for_prediction)
predictions = predictor.predict_proba(test_data, model=model_for_prediction)

# delete these extra models so they don't affect rest of tutorial
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)  

Deleting model WeightedEnsemble_L2Best. All files under agModels-predict_porto_2/models/WeightedEnsemble_L2Best/ will be removed.


### Collapsing bagged ensembles via `refit_full`

For an ensemble predictor trained with bagging (as done above), recall there ~10 bagged copies of each individual model trained on different train/validation folds. We can collapse this bag of ~10 models into a single model that’s fit to the full dataset, which can greatly reduce its memory/latency requirements (but may also reduce accuracy). Below we refit such a model for each original model but you can alternatively do this for just a particular model by specifying the model argument of `refit_full()`.

In [57]:
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data, silent=True)

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	1.49s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	0.09s	 = Training   runtime
Updated best model to "WeightedEnsemble_L2_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "WeightedEnsemble_L2_FULL" for predict() and predict_proba().
Refit complete, total runtime = 2.57s


Name of each refit-full model corresponding to a previous bagged ensemble:
{'LightGBM_BAG_L1': 'LightGBM_BAG_L1_FULL', 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.618328,0.623974,0.824968,0.744501,4.017492,0.824968,0.744501,4.017492,1,True,1
1,WeightedEnsemble_L2,0.618328,0.623974,0.830042,0.834036,4.108663,0.005074,0.089535,0.091171,2,True,2
2,LightGBM_BAG_L1_FULL,0.616898,NaN,0.106917,NaN,1.491984,0.106917,NaN,1.491984,1,True,3
3,WeightedEnsemble_L2_FULL,0.616898,NaN,0.112218,NaN,1.583154,0.005301,NaN,0.091171,2,True,4


This adds the refit-full models to the leaderboard and we can opt to use any of them for prediction just like any other model. Note `pred_time_test` and `pred_time_val` list the time taken to produce predictions with each model (in seconds) on the test/validation data. Since the refit-full models were trained using all of the data, there is no internal validation score (`score_val`) available for them. You can also call `refit_full()` with non-bagged models to refit the same models to your full dataset (there won’t be memory/latency gains in this case but test accuracy may improve).

### Model distillation

While computationally-favorable, single individual models will usually have lower accuracy than weighted/stacked/bagged ensembles. Model Distillation offers one way to retain the computational benefits of a single model, while enjoying some of the accuracy-boost that comes with ensembling. 

"Fast, Accurate, and Simple Models for Tabular Data via Augmented Distillation": https://arxiv.org/abs/2006.14284

The idea is to train the individual model (which we can call the student) to mimic the predictions of the full stack ensemble (the teacher). Like `refit_full()`, the `distill()` function will produce additional models we can opt to use for prediction.

In [58]:
student_models = predictor.distill(time_limit=30)  # specify much longer time limit in real applications
print(student_models)

# preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
preds_student = predictor.predict_proba(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)

Distilling with teacher='WeightedEnsemble_L2_FULL', teacher_preds=soft, augment_method=spunge ...
SPUNGE: Augmenting training data with 100000 synthetic samples for distillation...
Distilling with each of these student models: ['LightGBM_DSTL', 'NeuralNetMXNet_DSTL', 'CatBoost_DSTL', 'RandomForestMSE_DSTL', 'NeuralNetTorch_DSTL']
Fitting 5 L1 models ...
Fitting model: LightGBM_DSTL ... Training model for up to 30.0s of the 30.0s of remaining time.
	Note: model has different eval_metric than default.
	-0.0337	 = Validation score   (-mean_squared_error)
	2.62s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetMXNet_DSTL ... Training model for up to 27.27s of the 27.26s of remaining time.
		Unable to import dependency mxnet. A quick tip is to install via `pip install mxnet --upgrade`, or `pip install mxnet_cu101 --upgrade`
Fitting model: CatBoost_DSTL ... Training model for up to 26.86s of the 26.86s of remaining time.
	Note: model has different eval_metric than d

['LightGBM_DSTL', 'CatBoost_DSTL', 'RandomForestMSE_DSTL', 'WeightedEnsemble_L2_DSTL']
predictions from LightGBM_DSTL: [0, 1]
                      model  score_test  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0             CatBoost_DSTL    0.630523   0.595678        0.170604       0.027418   18.768441                 0.170604                0.027418          18.768441            1       True          6
1  WeightedEnsemble_L2_DSTL    0.626658   0.599191        1.228356       0.172532  137.877636                 0.021181                0.000668           0.155618            2       True          8
2           LightGBM_BAG_L1    0.618328   0.623974        0.809814       0.744501    4.017492                 0.809814                0.744501           4.017492            1       True          1
3       WeightedEnsemble_L2    0.618328   0.623974        0.816011       0.834036    4

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_DSTL,0.630523,0.595678,0.170604,0.027418,18.768441,0.170604,0.027418,18.768441,1,True,6
1,WeightedEnsemble_L2_DSTL,0.626658,0.599191,1.228356,0.172532,137.877636,0.021181,0.000668,0.155618,2,True,8
2,LightGBM_BAG_L1,0.618328,0.623974,0.809814,0.744501,4.017492,0.809814,0.744501,4.017492,1,True,1
3,WeightedEnsemble_L2,0.618328,0.623974,0.816011,0.834036,4.108663,0.006197,0.089535,0.091171,2,True,2
4,LightGBM_DSTL,0.617487,0.585840,0.088338,0.021080,2.617640,0.088338,0.021080,2.617640,1,True,5
5,LightGBM_BAG_L1_FULL,0.616898,NaN,0.095858,NaN,1.491984,0.095858,NaN,1.491984,1,True,3
6,WeightedEnsemble_L2_FULL,0.616898,NaN,0.100391,NaN,1.583154,0.004533,NaN,0.091171,2,True,4
7,RandomForestMSE_DSTL,0.592676,0.567056,0.948233,0.123366,116.335937,0.948233,0.123366,116.335937,1,True,7


### Faster presets or hyperparameters

Instead of trying to speed up a cumbersome trained model at prediction time, if you know inference latency or memory will be an issue at the outset, then you can adjust the training process accordingly to ensure `fit()` does not produce unwieldy models.

One option is to specify more lightweight presets:

In [59]:
presets = ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20230702_174411/"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Presets specified: ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "AutogluonModels/ag-20230702_174411/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify 

Another option is to specify more lightweight hyperparameters:

In [60]:
# time_limit=30 -> 3600

predictor_light = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data, 
    hyperparameters='very_light', 
    time_limit=3600
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230702_175205/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230702_175205/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    152776.55 MB
	T

Here you can set hyperparameters to either 'light', 'very_light', or 'toy' to obtain progressively smaller (but less accurate) models and predictors. Advanced users may instead try manually specifying particular models’ hyperparameters in order to make them faster/smaller.

Finally, you may also exclude specific unwieldy models from being trained at all. Below we exclude models that tend to be slower (K Nearest Neighbors, Neural Network, models with custom larger-than-default hyperparameters):

In [61]:
# time_limit=30 -> 3600 
    
excluded_model_types = ['KNN', 'NN', 'custom']

predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, 
    excluded_model_types=excluded_model_types, 
    time_limit=3600
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230702_180857/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230702_180857/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    152689.08 MB
	T

## If you encounter memory issues

To reduce memory usage during training, you may try each of the following strategies individually or combinations of them (these may harm accuracy):

- In `fit()`, set `num_bag_sets=1` (can also try values greater than 1 to harm accuracy less)

- In `fit()`, set `excluded_model_types=['KNN', 'XT', 'RF']` (or some subset of these models)

- Try different `presets` in `fit()`

- In `fit()`, set `hyperparameters='light'` or `hyperparameters='very_light'`

- Text fields in your table require substantial memory for N-gram featurization. To mitigate this in `fit()`, you can either:
    + (1) add `'ignore_text'` to your `presets` list (to ignore text features), or 
    + (2) specify the argument:

```
from sklearn.feature_extraction.text import CountVectorizer
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

feature_generator = AutoMLPipelineFeatureGenerator(
    vectorizer=CountVectorizer(
        min_df=30, 
        ngram_range=(1, 3), 
        max_features=MAX_NGRAM, 
        dtype=np.uint8
    )
)
```

where `MAX_NGRAM=1000` say (try various values under 10000 to reduce the number of N-gram features used to represent each text field).

In addition to reducing memory usage, many of the above strategies can also be used to reduce training times.

To reduce memory usage during inference:

- If trying to produce predictions for a large test dataset, break the test data into smaller chunks as demonstrated in FAQ.
    + https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-faq.html#sec-faq

- If models have been previously persisted in memory but inference-speed is not a major concern, call `predictor.unpersist_models()`

- If models have been previously persisted in memory, bagging was used in `fit()`, and inference-speed is a concern: call `predictor.refit_full()` and use one of the refit-full models for prediction (ensure this is the only model persisted in memory)

## If you encounter disk space issues

To reduce disk usage, you may try each of the following strategies individually or combinations of them:

- Make sure to delete all `predictor.path` folders from previous `fit()` runs
    + These can eat up your free space if you call `fit()` many times
    + If you didn’t specify path, AutoGluon still automatically saved its models to a folder called: `“AutogluonModels/ag-[TIMESTAMP]”`, where TIMESTAMP records when `fit()` was called, so make sure to also delete these folders if you run low on free space

- Call `predictor.save_space()` to delete auxiliary files produced during `fit()`

- Call `predictor.delete_models(models_to_keep='best', dry_run=False)` if you only intend to use this predictor for inference going forward (will delete files required for non-prediction-related functionality like `fit_summary`)

- In `fit()`, you can add `'optimize_for_deployment'` to the presets list, which will automatically invoke the previous two strategies after training

Most of the above strategies to reduce memory usage will also reduce disk usage (but may harm accuracy).

## References

The following paper describes how AutoGluon internally operates on tabular data:

Erickson et al. "AutoGluon-Tabular: Robust and Accurate AutoML for Structured Data". Arxiv, 2020: https://arxiv.org/abs/2003.06505